In [1]:
from dataset import create_dataset, create_sampler, create_loader
from models.model_nlvr import XVLM
from models.model_cosmos import XVLM
from models import XVLMBase, build_mlp, load_pretrained
config = {'train_file': ['data/finetune/nlvr_train.json'], 'val_file': ['data/finetune/nlvr_dev.json'], 
'test_file': ['data/finetune/nlvr_test.json'], 'image_root': 'images/nlvr2/', 
'vision_config': 'configs/config_swinB_384.json', 'use_clip_vit': False, 
'use_swin': True, 'image_res': 384, 'patch_size': 32, 'use_roberta': False, 
'text_config': 'configs/config_bert.json', 'text_encoder': 'data/bert-base-uncased', 
'batch_size': 5, 'optimizer': {'opt': 'adamW', 'lr': 3e-05, 'weight_decay': 0.01, 'lr_mult': 2}, 'schedular': {'sched': 'linear', 'lr': 3e-05, 'epochs': 10, 'num_warmup_steps': 0.1}}
# config_vqa = {'train_file': ['data/finetune/vqa_train.json', 'data/finetune/vqa_val.json', 'data/finetune/vg_qa.json'], 'test_file': ['data/finetune/vqa_test.json'], 'answer_list': 'data/finetune/answer_list.json', 'vqa_root': 'images/coco/', 'vg_root': 'images/visualgenome/', 'vision_config': 'configs/config_swinB_384.json', 'use_clip_vit': False, 'use_swin': True, 'image_res': 384, 'patch_size': 32, 'use_roberta': False, 'text_config': 'configs/config_bert.json', 'text_encoder': 'data/bert-base-uncased', 'num_dec_layers': 6, 'batch_size_train': 24, 'batch_size_test': 32, 'max_tokens': 40, 'k_test': 128, 'optimizer': {'opt': 'adamW', 'lr': 5e-05, 'weight_decay': 0.01, 'lr_mult': 2}, 'schedular': {'sched': 'linear', 'lr': 5e-05, 'epochs': 10, 'num_warmup_steps': 0.1}, 'start_eval': 7}
config = {'train_file': ['data/finetune/cosmos_train.json'], 'val_file': ['data/finetune/cosmos_val.json'], 
'test_file': ['data/finetune/cosmos_test.json'], 'image_root': 'images/cosmos/', 
'vision_config': 'configs/config_swinB_384.json', 'use_clip_vit': False, 
'use_swin': True, 'image_res': 384, 'patch_size': 32, 'use_roberta': False, 
'text_config': 'configs/config_bert.json', 'text_encoder': 'data/bert-base-uncased', 
'batch_size': 5, 'optimizer': {'opt': 'adamW', 'lr': 3e-05, 'weight_decay': 0.01, 'lr_mult': 2}, 'schedular': {'sched': 'linear', 'lr': 3e-05, 'epochs': 10, 'num_warmup_steps': 0.1}}
train_dataset, val_dataset, test_dataset = create_dataset('cosmos', config);
datasets = [train_dataset, val_dataset, test_dataset]
# config_vqa['pad_token_id'] = train_dataset.pad_token_id
# config_vqa['eos'] = train_dataset.eos_token

/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:834: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [2]:
train_loader, val_loader, test_loader = create_loader(datasets, [None, None, None], batch_size=[config['batch_size']] * 3,
                                                          num_workers=[4, 4, 4], is_trains=[True, False, False],
                                                          collate_fns=[None, None, None])

In [3]:
len(test_dataset)

1700

In [4]:
import torch
torch.cuda.set_device(1) 

In [5]:
model = XVLM(config=config)
model.load_pretrained('output/cosmos/checkpoint_best.pth', config, is_eval=False)
model = model.to('cuda');

### Loading pretrained vision encoder
### Loading pretrained text encoder
load checkpoint from output/cosmos/checkpoint_best.pth
missing_keys:  []
unexpected_keys:  []


In [6]:
from models.tokenization_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['text_encoder'])

In [7]:
s='''
Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside.
Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside.
Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside.
Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside.
Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside. Twenty-five years Dana had been waiting. She tried to be patient during that time but she hadn't always managed to be as patient as she'd like. But today the opportunity had finally come. The thing she always imagined would make her the happiest person in the world was about to happen. She didn't know why at this specific time she all of a sudden felt sick inside.
'''
#s='Hello my name is funny hell and I really hate you'
len(s)

5506

In [7]:
from tqdm import tqdm

In [8]:
device='cuda'
prediction = []
for image, caption1, caption2, targets in tqdm(test_loader):
    image, targets = image.to(device), targets.to(device)   
    text_inputs_1 = tokenizer(caption1, padding='longest', return_tensors="pt",max_length=512, truncation=True).to(device)
    text_inputs_2 = tokenizer(caption2, padding='longest', return_tensors="pt",max_length=512, truncation=True).to(device)

    loss = model(image, text_inputs_1.input_ids, text_inputs_1.attention_mask, 
    text_inputs_2.input_ids, text_inputs_2.attention_mask, 
    targets=targets, train=False)
    prediction += loss.argmax(dim=1).tolist()
    # break

100%|██████████| 340/340 [00:39<00:00,  8.64it/s]


In [9]:
import pandas as pd
df = pd.DataFrame(columns=['caption1','caption2','target'])
for image, caption1, caption2, targets in tqdm(test_loader):
    # df = df.append([caption1, caption2, targets])
    for c1,c2,t in zip(caption1, caption2, targets):
        df.loc[len(df)] = [c1,c2,t.item()]

  0%|          | 0/340 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████| 340/340 [00:08<00:00, 37.86it/s]


In [10]:
df['predict'] = prediction

In [11]:
confusion_matrix = pd.crosstab(df['target'], df['predict'], rownames=['target'], colnames=['predict'])
print (confusion_matrix)

predict    0    1
target           
0        768   82
1        522  328


In [17]:
pd.options.display.max_colwidth = 500

0       False
1       False
2       False
3        True
4       False
        ...  
1695     True
1696    False
1697    False
1698    False
1699     True
Length: 1700, dtype: bool

In [27]:
import pyarrow as pa
entail = pa.ipc.RecordBatchFileReader(
    pa.memory_map(f"entail.arrow", "r")
).read_all().to_pandas()

In [33]:
df['entail'] = entail['score']

In [48]:
df[(df['target']==1) & (df['predict']==0)]

,caption1,caption2,target,predict,entail
3,"Actor, musician, director and devoted follower of Christ, PERSON, has been stricken with an extremely rare form of what can only be described as a biblical disease.",A shocking report about the former child actor PERSON was just another hoax from a well-known purveyor of satire.,1,0,neutral
6,A bird (variously described as a goose or a duck) helping to keep a puppy warm,"It isn’t unprecedented for birds and dogs to strike up friendships, thus claim is false",1,0,neutral
7,"A photograph shows PERSON, a white woman who was killed by a black police officer in GPE.",A viral meme got key facts wrong while attempting to make a point about police violence and racial injustice in the GPE,1,0,neutral
23,A video shows people in GPE tearing down a CARDINALG tower in an attempt to stop the spread of COVID-19 coronavirus disease.,The video shows protesters in GPE in DATE (before the COVID-19 pandemic) tearing down a surveillance tower.,1,0,neutral
25,ORG warned that Asteroid 52768 (DATE) could hit LOC in DATE and cause catastrophic danger.,"This image of asteroid PRODUCT, taken by ORG's ORG on DATE, shows the north polar region highlighting the major physiographic features of the northern hemisphere; the saddle seen at the bottom.",1,0,neutral
...,...,...,...,...,...
1686,"A photo shows PERSON laughing after PERSON ""pranked"" her by disguising himself as a palace guard.",The prince and queen shortly before a ceremonial review of ORG at FAC.,1,0,contradiction
1688,"ORG secretly bought the “WORK_OF_ART,” PERSON, from a reclusive GPE businessman, PERSON, for MONEY in DATE.",The reports related to Mark purchasing a yacht are completely inaccurate as he did not purchase a yacht.,1,0,contradiction
1692,A video shows a naked child using bedsheets to escape from an upper story of FAC.,The naked man was actually escaping from Princess Eleanor’s (PERSON) chambers,1,0,contradiction
1695,President PERSON trademarked the name 'WORK_OF_ART' and receives royalties every time the term is used.,There was no truth that PERSON family MONEY with ORG,1,0,neutral


In [7]:
torch.cat([torch.tensor([5,2]), torch.tensor([5,2])],dim=-1)

tensor([5, 2, 5, 2])

In [9]:
loss

tensor([[ 0.0831, -0.3668],
        [ 0.0402, -0.0914],
        [-0.0596, -0.3729],
        [-0.2012, -0.3863],
        [-0.0547, -0.1801]], device='cuda:1', grad_fn=<AddmmBackward>)

In [5]:
device='cuda'
for (image0, image1, text, targets) in train_loader:
    images = torch.cat([image0, image1], dim=0)
    images, targets = images.to(device), targets.to(device)   
    
    text_inputs = tokenizer(text, padding='longest', return_tensors="pt").to(device)  
    
    loss = model(images, text_inputs.input_ids, text_inputs.attention_mask, targets=targets, train=True)
    break

NameError: name 'torch' is not defined

In [7]:
from models.xbert import BertConfig, BertForMaskedLM, BertModel
config_text = BertConfig.from_json_file(config['text_config'])
text_encoder = BertModel(config=config_text, add_pooling_layer=False)

In [8]:
image_embeds, image_atts = model.get_vision_embeds(images)

In [9]:
image_embeds.shape, image_atts.shape

(torch.Size([10, 145, 1024]), torch.Size([10, 145]))

In [10]:
image_embeds, image_atts = model.get_vision_embeds(images)
image0_embeds, image1_embeds = torch.split(image_embeds, targets.size(0))
(text_ids, text_atts) = text_inputs.input_ids, text_inputs.attention_mask
r = model.get_cross_embeds([image0_embeds, image1_embeds], 
    [image_atts[:image0_embeds.size(0)], image_atts[image0_embeds.size(0):]],
    text_ids=text_ids, text_atts=text_atts)

In [14]:
r[:, 0, :].shape

torch.Size([5, 768])

In [1]:
text_encoder, init = build_text_encoder(config, vision_width=145,  config_text=config_text)
#text_encoder = BertModel(config=config_text, add_pooling_layer=False)

NameError: name 'BertModel' is not defined

In [38]:
text_ids.shape, text_atts.shape, image_embeds.shape, image_atts.shape

(torch.Size([5, 19]),
 torch.Size([5, 19]),
 torch.Size([10, 145, 1024]),
 torch.Size([10, 145]))

In [49]:
b = text_encoder(text_ids,
            attention_mask=text_atts,
            encoder_hidden_states=[image0_embeds, image1_embeds],
            encoder_attention_mask=[image_atts[:image0_embeds.size(0)], image_atts[image0_embeds.size(0):]],
            return_dict=True,)

AttributeError: 'list' object has no attribute 'dim'

In [48]:
b.last_hidden_state.shape

torch.Size([5, 19, 768])